## Train a SentencePiece subword tokenizer for Tibetan
This code will train the SentencePiece subword tokenizer to build a BPE (Byte pair encoding) model

This code is adapted from an example in the Esukhia bonltk repository https://github.com/Esukhia/bonltk/blob/master/nbs/train_tokenizers/sentence_piece.ipynb. 

For the original SentencePiece paper, see https://www.aclweb.org/anthology/D18-2012/

All code below is licensed under Apache License 2.0 

## Import Libraries

In [61]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  folder_prefix = '/content/drive/MyDrive/Colab Notebooks/'
  temp_folder = '/tmp/'
  google_collab = True
else:
  folder_prefix = ''
  temp_folder = '/tmp/'
  google_collab = False


if google_collab:
    !pip install botok
    !pip install 'tqdm>=4.59.0' # make sure to use a sufficiently new version of tqdm that supports multithreading
    !pip install sentencepiece

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
# sentencepiece subword tokenizer
import sentencepiece as spm

# file access
import os
from glob import glob
from pathlib import Path 

# progress bar and multiprocessing
from tqdm.notebook import tqdm

# helper stuff 
import itertools
import traceback
import sys
from typing import List

## Configuration

In [49]:
model_name = 'bo_classical'
source_files = [folder_prefix+'data/tokenized/Corpora/BDRC/**/*.txt', folder_prefix+'data/tokenized/Corpora/Tibetan Dictionaries/**/*.txt']
corpus_file_tokenized   = Path(temp_folder)/'corpus_tokenized-all.txt'
corpus_file_untokenized = Path(temp_folder)/'corpus_untokenized-all.txt'
models_path  = Path(folder_prefix+'data/models/tokenizers')
os.makedirs(models_path, exist_ok=True)



## Merge Input Data into a Single File

In [50]:
# undo the pre-existing tokenization that was done with the botok library
def undo_tokenization(text):
    for f, t in [(' ', ''), ('_', ' ')]:
        text = text.replace(f, t)
    return text

# combine the entire corpus into a single file
def combine_files(source_files, out_file, remove_pretokenization=False):
    corpus = ''
    n_sentences = 0
    
    
    if out_file.is_file(): 
        return  # output file already exists -> do nothing
    
    # concatenate all files into a single one
    for path_pattern in source_files:
        for path in tqdm(glob(path_pattern, recursive=True)):
            text = Path(path).read_text()

            if remove_pretokenization: 
                text = undo_tokenization(text)

            n_sentences += text.count('\n')
            corpus += text + '\n'
        
    print('[INFO] No. sentences the corpus contains:', n_sentences)
    out_file.write_text(corpus)



In [51]:
# combine the entire corpus into a single file 

# 1. create merged file for the complete corpus with existing word tokenization from the botok library
combine_files(source_files, corpus_file_tokenized, remove_pretokenization = False)

# 2. create merged file for the complete corpus without tokenization
combine_files(source_files, corpus_file_untokenized, remove_pretokenization = True)

  0%|          | 0/24 [00:00<?, ?it/s]

0it [00:00, ?it/s]

[INFO] No. sentences the corpus contains: 244813


  0%|          | 0/24 [00:00<?, ?it/s]

0it [00:00, ?it/s]

[INFO] No. sentences the corpus contains: 244813


In [52]:
# !head {corpus_file_tokenized}

In [53]:
# !head {corpus_file_untokenized}

In [54]:
def train_sentencepiece_tokenizer(input_file, model_name, models_path, model_type='unigram', vocab_size=30000):
    model_prefix = f'{model_name}-{model_type}'
    
    if (models_path/f'{model_prefix}.model').is_file():
        return models_path/model_prefix

    spm.SentencePieceTrainer.Train(
    f'--input={input_file} \
      --model_prefix={model_prefix} \
      --vocab_size={vocab_size} \
      --model_type={model_type} \
      --character_coverage=0.9998 \
      --unk_id=0 \
      --pad_id=-1 \
      --bos_id=-1 \
      --eos_id=-1'
    )

    os.system(f'mv {model_prefix}.* "{models_path}"')
    print(f'mv {model_prefix}.* "{models_path}"')
    
    return models_path/model_prefix

## Train sentencepiece models for subword tokenization

### Create word-based model, based on pretokenized data that was previously processed by botok library

In [55]:
model_path = train_sentencepiece_tokenizer(corpus_file_tokenized, model_name, models_path, model_type='word')
sp = spm.SentencePieceProcessor()
sp.load(f'{model_path}.model')

#print(sp.encode_as_pieces('ཡིད་ དགའ་བ འི་ ལུས་ ཤིན་ཏུ་ སྦྱངས་པ ར་ འགྱུར་ རོ་ ཞེས་ འབྱུང་བ འི་ ཕྱིར་ རོ །_། ལུས་ མཉེན་པ ར་ བྱེད་པ་ ཞེས་ བྱ་བ་ ནི་ལ ས་ སུ་ རུང་བ ར་ བྱེད་པ འོ །_།'))
print(sp.encode_as_pieces('ཀྱང་ ཁ་ལོ་ སྒྱུར་བ་ ལ་ གསུངས་པ །_ ཁ་ལོ་ སྒྱུར་བ་ མི་སྐྱ་བོ་ རྦབ་ རྦབ་པོ་ རིད་པ་ ཉམ་ཆུང་བ་ དབང་པོ་__ ཉམས་པ །_ སྐྱེ་བོ་ མང་པོ འི་ མིག་ གིས་ ཀྱང་ ལྟ་ མི་ ཕོད་པ་ འདི་ ཅི་ ཡིན །_'))


['▁ཀྱང་', '▁ཁ་ལོ་', '▁སྒྱུར་བ་', '▁ལ་', '▁གསུངས་པ', '▁།_', '▁ཁ་ལོ་', '▁སྒྱུར་བ་', '▁མི་སྐྱ་བོ་▁རྦབ་▁རྦབ་པོ་▁རིད་པ་', '▁ཉམ་ཆུང་བ་', '▁དབང་པོ་__', '▁ཉམས་པ', '▁།_', '▁སྐྱེ་བོ་', '▁མང་པོ', '▁འི་', '▁མིག་', '▁གིས་', '▁ཀྱང་', '▁ལྟ་', '▁མི་', '▁ཕོད་པ་', '▁འདི་', '▁ཅི་', '▁ཡིན', '▁།_']


### BPE (Byte pair encoding) model

In [56]:
model_path = train_sentencepiece_tokenizer(corpus_file_untokenized, model_name, models_path, model_type='bpe')

sp = spm.SentencePieceProcessor()
sp.load(f'{model_path}.model')

print(sp.encode_as_pieces('ཀྱང་ཁ་ལོ་སྒྱུར་བ་ལ་གསུངས་པ། ཁ་ལོ་སྒྱུར་བ་མི་སྐྱ་བོ་རྦབ་རྦབ་པོ་རིད་པ་ཉམ་ཆུང་བ་དབང་པོ་ཉམས་པ། སྐྱེ་བོ་མང་པོའི་མིག་གིས་ཀྱང་ལྟ་མི་ཕོད་པ་འདི་ཅི་ཡིན།'))

['▁ཀྱང་', 'ཁ་ལོ་སྒྱུར་བ', '་ལ་ག', 'སུ', 'ངས་པ', '།', '▁ཁ', '་ལ', 'ོ་སྒྱུར་བ', '་མི་སྐྱ', '་བ', 'ོ་ར', 'ྦ', 'བ', '་ར', 'ྦ', 'བ་པ', 'ོ་རི', 'ད་པ', '་ཉ', 'མ་', 'ཆུང་བ', '་དབང་', 'པོ་', 'ཉམས་པ', '།', '▁སྐྱེ་བོ་མང་པོ', 'འི་མིག', '་གིས་', 'ཀྱང་', 'ལྟ', '་མི་', 'ཕ', 'ོད་པ', '་འདི་', 'ཅི་', 'ཡིན།']


### Create unigram model with sentencepiece

In [57]:
model_path = train_sentencepiece_tokenizer(corpus_file_untokenized, model_name, models_path, model_type='unigram')

sp = spm.SentencePieceProcessor()
sp.load(f'{model_path}.model')

print(sp.encode_as_pieces('ཀྱང་ཁ་ལོ་སྒྱུར་བ་ལ་གསུངས་པ། ཁ་ལོ་སྒྱུར་བ་མི་སྐྱ་བོ་རྦབ་རྦབ་པོ་རིད་པ་ཉམ་ཆུང་བ་དབང་པོ་ཉམས་པ། སྐྱེ་བོ་མང་པོའི་མིག་གིས་ཀྱང་ལྟ་མི་ཕོད་པ་འདི་ཅི་ཡིན།'))

['▁ཀྱང་', 'ཁ་ལོ་སྒྱུར་བ་', 'ལ་', 'གསུངས་པ', '།', '▁', 'ཁ་ལོ་སྒྱུར་བ་', 'མི་', 'སྐྱ', '་བོ', '་', 'ར', 'ྦ', 'བ་', 'ར', 'ྦ', 'བ་', 'པོ་', 'རི', 'ད་པ་', 'ཉམ་ཆུང་', 'བ་', 'དབང་པོ་', 'ཉམས་པ', '།', '▁', 'སྐྱེ་བོ་མང་པོའི་', 'མིག་གིས་', 'ཀྱང་', 'ལྟ་', 'མི་ཕོད', '་པ་', 'འདི་', 'ཅི་', 'ཡིན།']


## Cleanup

In [59]:
# delete the combined files with the entire corpus
corpus_file_tokenized.unlink()
corpus_file_untokenized.unlink()


FileNotFoundError: ignored